In [99]:
import numpy as np
np.set_printoptions(suppress=True)

import pandas as pd

from IPython.display import display, HTML

from sklearn.neighbors import NearestNeighbors

In [2]:
npz = np.load("E:/Data/MIR/EU_SOUNDS_FEATURES/combined_features.npz")

In [45]:
audio_template = "<audio controls><source controls src=\"file:///{0}\" type=\"audio/mp3\"></audio>"
print audio_template.format("D:/Research/Data/MIR/MVD/Audio/MV-VIS/Metal/Metal_1.mp3")
HTML( audio_template.format("D:/Research/Data/MIR/MVD/Audio/MV-VIS/Metal/Metal_1.mp3"))

<audio controls><source controls src="file:///D:/Research/Data/MIR/MVD/Audio/MV-VIS/Metal/Metal_1.mp3" type="audio/mp3"></audio>


In [51]:
from sklearn.preprocessing import StandardScaler, Normalizer

In [49]:
ssd_scaled = StandardScaler().fit_transform(ssd)

In [52]:
ssd_norm = Normalizer().fit_transform(ssd)

In [54]:
ssd_norm.max(axis=0).sum()

61.898535826040508

In [90]:
ssd_my_norm = npz["ssd"]
ssd_my_norm -= ssd_my_norm.min(axis=0)
ssd_my_norm /= ssd_my_norm.max(axis=0)
ssd_my_norm /= ssd_my_norm.shape[1]

In [92]:
my_rh = npz["rh"]
my_rh /= my_rh.sum(axis=1).reshape((my_rh.shape[0],1))
my_rh /= 60.0

my_rp = npz["rp"]
my_rp /= my_rp.sum(axis=1).reshape((my_rp.shape[0],1))
my_rp /= 1440.0

In [204]:
comb3 = np.concatenate([ssd_my_norm * 0.99,
                        my_rp * 0.99,
                        my_rh * 0.99], axis=1)

nn = NearestNeighbors(n_neighbors=10, algorithm='brute', metric='cosine').fit(comb3)

Vögel 14000
Latin 24000
Jeweish 2000

In [210]:
query_idx = 2000

dist, neighbors = nn.kneighbors(comb3[query_idx,:],return_distance=True)

html = "<table>"

for i in range(neighbors[0].shape[0]):
    html = "{0}<tr><td>{1}</td><td>{2}</td><td>{3}</td></tr>".format(html, 
                                       audio_template.format(npz["filenames"][neighbors[0][i]].replace("\\","/").replace("_FEATURES","").replace(".npz","")),
                                                        neighbors[0][i],
                                                        dist[0][i])

html = "%s</table>" % (html)

#print html
HTML(html)

,2000,1.99840144433e-15
,1875,0.00967544550142
,2484,0.0146186604307
,1196,0.0151140865138
,2120,0.019522467382
,3026,0.0202196077814
,791,0.0202852117887
,1357,0.0208928827672
,1970,0.0212953024719
,27581,0.0213724603826


In [166]:
import scipy

In [211]:
query_space = np.zeros(tuple(comb3.shape))
query_space[:,:] = comb3[query_idx,:]
res = scipy.stats.entropy(comb3.T, qk=query_space.T, base=None)

neighbors = np.argsort(res)
dist = res[neighbors]

html = "<table>"

for i in range(10):
    html = "{0}<tr><td>{1}</td><td>{2}</td><td>{3}</td></tr>".format(html, 
                                       audio_template.format(npz["filenames"][neighbors[i]].replace("\\","/").replace("_FEATURES","").replace(".npz","")),
                                                        neighbors[i],
                                                        dist[i])

html = "%s</table>" % (html)

#print html
HTML(html)

,2000,0.0
,1875,0.0232595219255
,2484,0.0234809289996
,1196,0.0281123177928
,2120,0.0293875030899
,791,0.0315123254678
,2062,0.0335980941881
,5980,0.034220829308
,2210,0.0349869005985
,99024,0.035010478591
